# This file is used to retrieve data and clean it!!

In [1]:
import refinitiv.data as rd
import refinitiv.data.eikon as ek
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings("ignore")

/Users/sambhavjain/Desktop/Ref_Hackathon/ESG_PortOpt/.conda/lib/python3.11/site-packages/refinitiv/data/eikon/__init__.py:15:FutureWarning: The refinitiv.data.eikon module will be removed in future library version v2.0. Please install and use the 'eikon' Python library instead or migrate your code to the Refinitiv/LSEG Data Library


In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x123a33c10 {name='workspace'}>

In [3]:
# Get constituents of the FTSE 100 index
constituents_data = ek.get_data(
    instruments=['0#.FTSE'],
    fields=['TR.CommonName', 'TR.GICSSectorName', 'TR.PrimaryQuote']
)[0]  # [0] to get the DataFrame from the result tuple

print(constituents_data)

   Instrument              Company Common Name Primary Quote RIC
0      STAN.L           Standard Chartered PLC            STAN.L
1      CRDA.L          Croda International PLC            CRDA.L
2      ANTO.L                  Antofagasta PLC            ANTO.L
3       EZJ.L                      Easyjet PLC             EZJ.L
4      BNZL.L                        Bunzl plc            BNZL.L
..        ...                              ...               ...
95     ULVR.L                     Unilever PLC            ULVR.L
96     OCDO.L                  Ocado Group PLC            OCDO.L
97     LSEG.L  London Stock Exchange Group PLC            LSEG.L
98     TSCO.L                        Tesco PLC            TSCO.L
99     LGEN.L        Legal & General Group PLC            LGEN.L

[100 rows x 3 columns]


In [4]:
ind_const, err = ek.get_data(
        instruments = constituents_data['Primary Quote RIC'].tolist(),
        fields = ['TR.CommonName','TR.CompanyMarketCap', "TR.TRBCIndustryGroup", "TR.TRESGScore"])

ind_const

,Instrument,Company Common Name,Company Market Cap,TRBC Industry Group Name,ESG Score
0,STAN.L,Standard Chartered PLC,19922771810.102001,Banking Services,91.992196
1,CRDA.L,Croda International PLC,6397417019.73823,Chemicals,80.448773
2,ANTO.L,Antofagasta PLC,22795655726.825199,Metals & Mining,77.346326
3,EZJ.L,Easyjet PLC,3518338945.87129,Passenger Transportation Services,54.559892
4,BNZL.L,Bunzl plc,10092518696.7535,Diversified Industrial Goods Wholesale,48.196805
...,...,...,...,...,...
95,ULVR.L,Unilever PLC,106534393669.759995,Personal & Household Products & Services,89.380916
96,OCDO.L,Ocado Group PLC,3408499048.26159,Diversified Retail,67.810071
97,LSEG.L,London Stock Exchange Group PLC,48734038969.854698,Investment Banking & Investment Services,75.824323
98,TSCO.L,Tesco PLC,21778728876.029999,Food & Drug Retailing,79.19792


In [5]:
# Applying the function
top_per_sector = ind_const.sort_values(by=['Company Market Cap', 'ESG Score'], ascending=False).head(100)

top_per_sector.reset_index(drop=True, inplace=True)
top_per_sector

,Instrument,Company Common Name,Company Market Cap,TRBC Industry Group Name,ESG Score
0,AZN.L,AstraZeneca PLC,186564632966.979004,Pharmaceuticals,95.504923
1,SHEL.L,Shell PLC,176852200585.119995,Oil & Gas,93.477656
2,HSBA.L,HSBC Holdings PLC,129120793425.919998,Banking Services,78.011392
3,ULVR.L,Unilever PLC,106534393669.759995,Personal & Household Products & Services,89.380916
4,RIO.L,Rio Tinto PLC,95121752499.619202,Metals & Mining,70.216734
...,...,...,...,...,...
95,FRAS.L,Frasers Group PLC,3696871464.17589,Specialty Retailers,47.595611
96,RS1R.L,RS Group PLC,3573725875.14518,Electronic Equipment & Parts,60.282361
97,EZJ.L,Easyjet PLC,3518338945.87129,Passenger Transportation Services,54.559892
98,OCDO.L,Ocado Group PLC,3408499048.26159,Diversified Retail,67.810071


In [6]:
ric = top_per_sector["Instrument"].to_list()

In [7]:
# Split the RIC list into 4 chunks
chunk_size = len(ric) // 4
ric_chunks = [ric[i:i + chunk_size] for i in range(0, len(ric), chunk_size)]

# Initialize an empty DataFrame to store the portfolio data
portfolio = pd.DataFrame()

# Fetch data for each chunk
for chunk in ric_chunks:
    for r in chunk:
        try:
            data, err = ek.get_data(r, fields=[ek.TR_Field('tr.close'), ek.TR_Field('tr.close.date')],
                                    parameters={"SDate": "2000-01-01", "EDate": "2024-05-01"})
            if not isinstance(data, pd.DataFrame):
                data = pd.DataFrame(data)
            
            # Append the data to the portfolio DataFrame
            portfolio = pd.concat([portfolio, data], ignore_index=True)
        except ek.RDError as e:
            print(f"Error fetching data for {r}: {e.message} (Error code: {e.code})")

# Optional: Output the final portfolio DataFrame to inspect the result
print(portfolio)

       Instrument  Price Close                  Date
0           AZN.L  2392.339763  2000-01-04T00:00:00Z
1           AZN.L  2346.502026  2000-01-05T00:00:00Z
2           AZN.L  2341.625671  2000-01-06T00:00:00Z
3           AZN.L  2397.216118  2000-01-07T00:00:00Z
4           AZN.L  2463.534546  2000-01-10T00:00:00Z
...           ...          ...                   ...
614804      SJP.L        421.0  2024-04-25T00:00:00Z
614805      SJP.L        435.4  2024-04-26T00:00:00Z
614806      SJP.L        444.8  2024-04-29T00:00:00Z
614807      SJP.L        435.6  2024-04-30T00:00:00Z
614808      SJP.L        433.2  2024-05-01T00:00:00Z

[614809 rows x 3 columns]


In [8]:
portfolio.groupby('Instrument').count()

,Price Close,Date
Instrument,,
AAF.L,1223,6147
AAL.L,6147,6147
ABF.L,6147,6147
ADML.L,4953,6147
AHT.L,6147,6147
...,...,...
UU.L,6147,6147
VOD.L,6147,6147
WEIR.L,6147,6147


In [9]:

# Convert 'Date' to datetime and normalize timezone if needed (remove timezone)
portfolio['Date'] = pd.to_datetime(portfolio['Date']).dt.tz_localize(None)

# Creating a full DataFrame assuming all combinations should exist
dates = pd.date_range(start='2000-01-01', end='2024-05-01', freq='D')
instruments = portfolio['Instrument'].unique()
full_index = pd.MultiIndex.from_product([dates, instruments], names=['Date', 'Instrument'])
full_df = pd.DataFrame(index=full_index).reset_index()

# Convert 'Date' in full_df to datetime and normalize timezone (since it's already without timezone, this is for demonstration)
full_df['Date'] = pd.to_datetime(full_df['Date'])

# Merge to find and handle missing data
merged_df = full_df.merge(portfolio, on=['Date', 'Instrument'], how='left')

# Check for missing data
missing_data = merged_df[merged_df['Price Close'].isna()]

# Step to handle duplicates: averaging 'Price Close' for same 'Date' and 'Instrument'
merged_df = merged_df.groupby(['Date', 'Instrument']).agg({'Price Close': 'mean'}).reset_index()

# Now pivot the DataFrame to the desired format
pivot_df = merged_df.pivot(index='Date', columns='Instrument', values='Price Close')

pivot_df = pivot_df.dropna(how='all')

# Displaying the first few rows of the pivoted DataFrame
pivot_df

Instrument,AAF.L,AAL.L,ABF.L,ADML.L,AHT.L,ANTO.L,AUTOA.L,AV.L,AZN.L,BAES.L,...,SVT.L,TSCO.L,TW.L,ULVR.L,UTG.L,UU.L,VOD.L,WEIR.L,WPP.L,WTB.L
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,<NA>,996.961318,327.61539,<NA>,154.593367,86.037092,<NA>,920.32412,2392.339763,408.5,...,625.338004,181.484681,88.529409,1013.333232,165.428281,529.350796,295.186056,231.0,970.0,538.21901
2000-01-05,<NA>,1005.684729,332.799959,<NA>,153.229308,86.037092,<NA>,893.154057,2346.502026,401.25,...,634.263903,184.480301,92.057755,1019.999898,165.428281,537.034921,280.914974,237.5,875.0,536.904143
2000-01-06,<NA>,1030.359522,364.401142,<NA>,151.410563,86.334798,<NA>,884.756038,2341.625671,404.75,...,682.56877,184.480301,94.303066,1053.333228,165.428281,557.952815,272.402399,249.0,867.5,543.04019
2000-01-07,<NA>,1066.74861,353.538236,<NA>,151.410563,86.334798,<NA>,904.516084,2397.216118,388.25,...,706.721203,181.983951,96.548378,1104.444334,165.428281,567.771419,287.67496,244.0,900.0,569.337536
2000-01-10,<NA>,1077.964425,357.241499,<NA>,146.863699,86.830975,<NA>,910.444097,2463.534546,380.25,...,699.370463,176.242345,93.34079,1064.444338,165.428281,577.163126,302.196411,243.5,943.5,552.244261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-25,108.5,2560.0,2686.0,2675.885688,5760.0,2227.0,688.6,458.5,12026.0,1320.0,...,2442.0,289.8,131.45,4082.0,917.0,1030.0,69.0,1967.0,796.4,3028.0
2024-04-26,109.8,2643.0,2635.0,2682.843786,6104.0,2255.0,695.4,463.8,11988.0,1340.0,...,2458.0,288.9,134.3,4133.0,932.0,1038.5,69.08,2010.0,812.4,3043.0
2024-04-29,112.1,2750.0,2654.0,2716.640262,5974.0,2282.0,696.6,467.9,12024.0,1355.0,...,2480.0,292.0,135.0,4105.0,933.5,1045.0,70.0,2042.0,817.2,3047.0


In [10]:
df_cleaned = pivot_df.dropna(how='all')
df_cleaned.isna().sum()

Instrument
AAF.L     4978
AAL.L       54
ABF.L       54
ADML.L    1248
AHT.L       54
          ... 
UU.L        54
VOD.L       54
WEIR.L      54
WPP.L       54
WTB.L       54
Length: 100, dtype: int64

In [11]:
# Drop rows where all elements are NaN
df_cleaned = pivot_df.dropna(how='all')

# Calculate the number of NaN values for each RIC
na_counts = df_cleaned.isna().sum()

# Filter out the RICs with more than 100 NaN values
ric_to_keep = na_counts[na_counts <= 100].index

# Create a new DataFrame with only the filtered RICs
filtered_pivot_df = pivot_df[ric_to_keep]

# Optional: Output the final filtered DataFrame to inspect the result
filtered_pivot_df.shape

(6201, 70)

In [12]:
# List of columns with missing values
columns_with_missing = filtered_pivot_df.columns[filtered_pivot_df.isna().any()].tolist()
print(f"Columns with missing values: {columns_with_missing}")

# Apply backward fill for initial missing values
df_bfill_initial = filtered_pivot_df[columns_with_missing].fillna(method='bfill')

# Apply interpolation for remaining missing values
df_filled = df_bfill_initial.interpolate(method='linear')

# Replace original columns with filled data
filtered_pivot_df[columns_with_missing] = df_filled

# Validate the result
print(filtered_pivot_df.isna().sum())

Columns with missing values: ['AAL.L', 'ABF.L', 'AHT.L', 'ANTO.L', 'AV.L', 'AZN.L', 'BAES.L', 'BARC.L', 'BATS.L', 'BDEV.L', 'BKGH.L', 'BNZL.L', 'BP.L', 'BT.L', 'CNA.L', 'CRDA.L', 'DCC.L', 'DGE.L', 'DPLM.L', 'EXPN.L', 'FCIT.L', 'GSK.L', 'HLMA.L', 'HSBA.L', 'HWDN.L', 'ICGIN.L', 'III.L', 'IMB.L', 'IMI.L', 'INF.L', 'KGF.L', 'LAND.L', 'LGEN.L', 'LLOY.L', 'MKS.L', 'NG.L', 'NWG.L', 'NXT.L', 'PRU.L', 'PSN.L', 'PSON.L', 'REL.L', 'RIO.L', 'RKT.L', 'RR.L', 'RS1R.L', 'RTO.L', 'SBRY.L', 'SDR.L', 'SGE.L', 'SGRO.L', 'SHEL.L', 'SJP.L', 'SMDS.L', 'SMIN.L', 'SMT.L', 'SN.L', 'SPX.L', 'SSE.L', 'STAN.L', 'SVT.L', 'TSCO.L', 'TW.L', 'ULVR.L', 'UTG.L', 'UU.L', 'VOD.L', 'WEIR.L', 'WPP.L', 'WTB.L']
Instrument
AAL.L     0
ABF.L     0
AHT.L     0
ANTO.L    0
AV.L      0
         ..
UU.L      0
VOD.L     0
WEIR.L    0
WPP.L     0
WTB.L     0
Length: 70, dtype: int64


In [13]:
portfolio_df = filtered_pivot_df

#export the data to a csv file within the same directory
portfolio_df.to_csv('portfolio.csv')

In [14]:
rd.close_session()